In [1]:
import os
from os.path import join
import numpy as np
import nibabel as nib
from utils import save2cifti
from 

In [11]:
work_dir = '/nfs/z1/userhome/zzl-xsk/Featurespace'
actavations_path = join(work_dir, 'prep/image_activations')
subs = ['sub-01','sub-02']
layers = ['googlenet-conv2_ds', 'googlenet-inception3a', 'googlenet-maxpool2']

In [12]:
for sub in subs:
    all_actvations = []
    for layer in layers:
        activations = np.load(join(actavations_path, f'{sub}_{layer}.npy'))
        all_actvations.append(activations)
    all_activations = np.concatenate(all_actvations, axis=1)
    np.save(join(actavations_path, f'{sub}_googlenet-concate.npy'), all_activations)

In [13]:
data = np.load('/nfs/z1/userhome/zzl-xsk/Featurespace/prep/image_activations/sub-02_googlenet-concate.npy')

#### 下面根据每个体素在三个层估计出的prf的R2来选择体素，组成一个新的prf

In [16]:
work_dir = '/nfs/z1/userhome/zzl-xsk/Featurespace'
cifti_path = '/nfs/z1/userhome/zzl-xsk/NOD/derivatives/ciftify'
prf_dir = join(work_dir, 'build/retinoparams/primaryvis-in-MMP')
corrmap_dir = join(work_dir, 'build/corrmap')
guass_path = join(work_dir, 'build/gaussianparams/primaryvis-in-MMP')
voxel_mask_path = os.path.join(work_dir, 'prep/voxel_masks')
map_dir = join(work_dir, 'anal/brainmap/masked_retinotopy')
mask_name = 'primaryvis-in-MMP'
subs = ['sub-01']
sub = subs[0]

In [17]:
voxel_mask_nii = nib.load(os.path.join(voxel_mask_path, f'nod-voxmask_{mask_name}.dlabel.nii'))
voxel_mask = voxel_mask_nii.get_fdata()

prf_data = nib.load(os.path.join(cifti_path, f'{sub}/results/ses-prf_task-prf/ses-prf_task-prf_params.dscalar.nii')).get_fdata()
#选取R2大于10的体素
R2_values = prf_data[3, :]
valid_R2_indices = np.where(R2_values >= 10)[0]

mmp_voxel_mask = np.squeeze(np.array(voxel_mask))
# 确定最终的mask indices
mmp_voxel_indices = np.where(mmp_voxel_mask==1)[0]
voxel_indices = np.intersect1d(mmp_voxel_indices, valid_R2_indices)

In [18]:
conv2_prf = np.load(join(prf_dir, f'{sub}_layer-googlenet-conv2_params.npy'),allow_pickle=True).item()
conv2_prf_R2 = conv2_prf['R2']

inception3a_prf = np.load(join(prf_dir, f'{sub}_layer-googlenet-inception3a_params.npy'),allow_pickle=True).item()
inception3a_prf_R2 = inception3a_prf['R2']

maxpool2_prf = np.load(join(prf_dir, f'{sub}_layer-googlenet-maxpool2_params.npy'),allow_pickle=True).item()
maxpool2_prf_R2 = maxpool2_prf['R2']


#### 使用R2 vote

In [19]:
new_prf = np.nan * np.zeros((4, 59412))
new_guass = { }
conv2_guass = np.load(join(guass_path, f'{sub}_layer-googlenet-conv2_Gauss.npy'),allow_pickle=True).item()
inception3a_guass = np.load(join(guass_path, f'{sub}_layer-googlenet-inception3a_Gauss.npy'),allow_pickle=True).item()
maxpool2_guass = np.load(join(guass_path, f'{sub}_layer-googlenet-maxpool2_Gauss.npy'),allow_pickle=True).item()

for voxel_idx in voxel_indices:

    r2_conv2 = conv2_prf_R2[voxel_idx]
    r2_inception3a = inception3a_prf_R2[voxel_idx]
    r2_maxpool2 = maxpool2_prf_R2[voxel_idx]

    max_r2 = np.nanmax([r2_conv2, r2_inception3a, r2_maxpool2])
    if max_r2 == r2_conv2:
        selected_prf = conv2_prf
        selected_guass = conv2_guass
    elif max_r2 == r2_inception3a:
        selected_prf = inception3a_prf
        selected_guass = inception3a_guass
    else:
        selected_prf = maxpool2_prf
        selected_guass = maxpool2_guass

    new_guass[voxel_idx] = selected_guass[voxel_idx]

    ecc = selected_prf['ecc'][voxel_idx]
    angle = selected_prf['ang'][voxel_idx]
    rfsize = selected_prf['rfsize'][voxel_idx]

    new_prf[0, voxel_idx] = ecc
    new_prf[1, voxel_idx] = angle
    new_prf[2, voxel_idx] = rfsize
    new_prf[3, voxel_idx] = max_r2


In [12]:
named_maps = ['ecc', 'ang', 'rfsize', 'R2']
bfiletemplate = nib.load('/nfs/z1/userhome/zzl-xsk/workingdir/template.dtseries.nii')
brain_models = bfiletemplate.header.get_index_map(1).brain_models
surface_brain_models = [bm for bm in brain_models if bm.model_type == 'CIFTI_MODEL_TYPE_SURFACE']
# print(pjoin(map_dir, sub, f'{sub}_masked-prior-prf.dscalar.nii'))
save2cifti(join(map_dir, sub, f'{sub}_voted-masked-dnn-prf.dscalar.nii'),  
            new_prf, surface_brain_models, map_names=named_maps)

In [13]:
#save guass params
np.save(join(guass_path, f'{sub}_voted_Gauss.npy'), np.array([new_guass]), allow_pickle=True)

#### 批处理

##### voting

In [20]:
work_dir = '/nfs/z1/userhome/zzl-xsk/Featurespace'
cifti_path = '/nfs/z1/userhome/zzl-xsk/NOD/derivatives/ciftify'
prf_dir = join(work_dir, 'build/retinoparams/primaryvis-in-MMP')
corrmap_dir = join(work_dir, 'build/corrmap')
guass_path = join(work_dir, 'build/gaussianparams/primaryvis-in-MMP')
voxel_mask_path = os.path.join(work_dir, 'prep/voxel_masks')
map_dir = join(work_dir, 'anal/brainmap/masked_retinotopy')
mask_name = 'primaryvis-in-MMP'
subs = [f'sub-0{isub}' for isub in range(1, 10)]

for sub in subs:
    #体素选择
    voxel_mask_nii = nib.load(os.path.join(voxel_mask_path, f'nod-voxmask_{mask_name}.dlabel.nii'))
    voxel_mask = voxel_mask_nii.get_fdata()
    prf_data = nib.load(os.path.join(cifti_path, f'{sub}/results/ses-prf_task-prf/ses-prf_task-prf_params.dscalar.nii')).get_fdata()
    #选取R2大于10的体素
    R2_values = prf_data[3, :]
    valid_R2_indices = np.where(R2_values >= 10)[0]
    mmp_voxel_mask = np.squeeze(np.array(voxel_mask))
    # 确定最终的mask indices
    mmp_voxel_indices = np.where(mmp_voxel_mask==1)[0]
    voxel_indices = np.intersect1d(mmp_voxel_indices, valid_R2_indices)
    #读取dnn prf 数值
    conv2_prf = np.load(join(prf_dir, f'{sub}_layer-googlenet-conv2_params.npy'),allow_pickle=True).item()
    conv2_prf_R2 = conv2_prf['R2']
    inception3a_prf = np.load(join(prf_dir, f'{sub}_layer-googlenet-inception3a_params.npy'),allow_pickle=True).item()
    inception3a_prf_R2 = inception3a_prf['R2']
    maxpool2_prf = np.load(join(prf_dir, f'{sub}_layer-googlenet-maxpool2_params.npy'),allow_pickle=True).item()
    maxpool2_prf_R2 = maxpool2_prf['R2']
    #
    new_prf = np.nan * np.zeros((4, 59412))
    new_guass = { }
    conv2_guass = np.load(join(guass_path, f'{sub}_layer-googlenet-conv2_Gauss.npy'),allow_pickle=True).item()
    inception3a_guass = np.load(join(guass_path, f'{sub}_layer-googlenet-inception3a_Gauss.npy'),allow_pickle=True).item()
    maxpool2_guass = np.load(join(guass_path, f'{sub}_layer-googlenet-maxpool2_Gauss.npy'),allow_pickle=True).item()

    for voxel_idx in voxel_indices:
        r2_conv2 = conv2_prf_R2[voxel_idx]
        r2_inception3a = inception3a_prf_R2[voxel_idx]
        r2_maxpool2 = maxpool2_prf_R2[voxel_idx]

        max_r2 = np.nanmax([r2_conv2, r2_inception3a, r2_maxpool2])
        if max_r2 == r2_conv2:
            selected_prf = conv2_prf
            selected_guass = conv2_guass
        elif max_r2 == r2_inception3a:
            selected_prf = inception3a_prf
            selected_guass = inception3a_guass
        else:
            selected_prf = maxpool2_prf
            selected_guass = maxpool2_guass

        new_guass[voxel_idx] = selected_guass[voxel_idx]

        ecc = selected_prf['ecc'][voxel_idx]
        angle = selected_prf['ang'][voxel_idx]
        rfsize = selected_prf['rfsize'][voxel_idx]

        new_prf[0, voxel_idx] = ecc
        new_prf[1, voxel_idx] = angle
        new_prf[2, voxel_idx] = rfsize
        new_prf[3, voxel_idx] = max_r2

    named_maps = ['ecc', 'ang', 'rfsize', 'R2']
    bfiletemplate = nib.load('/nfs/z1/userhome/zzl-xsk/workingdir/template.dtseries.nii')
    brain_models = bfiletemplate.header.get_index_map(1).brain_models
    surface_brain_models = [bm for bm in brain_models if bm.model_type == 'CIFTI_MODEL_TYPE_SURFACE']
    # print(pjoin(map_dir, sub, f'{sub}_masked-prior-prf.dscalar.nii'))
    save2cifti(join(map_dir, sub, f'{sub}_voted-masked-dnn-prf.dscalar.nii'),  
                new_prf, surface_brain_models, map_names=named_maps)
    np.save(join(guass_path, f'{sub}_voted_Gauss.npy'), np.array([new_guass]), allow_pickle=True)

##### weighted

In [176]:

def average_angle(weights, degrees):
    # 将输入角度从以正 y 轴为 0° 转换为以正 x 轴为 0°
    degrees = (degrees - 90) % 360
    degrees[degrees > 180] -= 360  # 转换到 [-180, 180) 范围
    
    radians = np.deg2rad(degrees)
    x = np.cos(radians)
    y = np.sin(radians)
    mean_x = np.nansum(weights*x)
    mean_y = np.nansum(weights*y)
    mean_angle = np.arctan2(mean_y, mean_x)
    mean_angle = np.rad2deg(mean_angle)

    # 将平均角度转换回以正 y 轴为 0° 的坐标系
    mean_angle = (mean_angle + 90) % 360
    mean_angle = (360 - mean_angle) % 360  # 使顺时针方向为正
    mean_angle = mean_angle - 360 if mean_angle > 180 else mean_angle  # 转换到 [-180, 180) 范围
    
    return -mean_angle

In [175]:
a = np.array([0.,-5., 8])
average_angle(a)

0.9978606740760938

In [159]:
def average_angle_fordnn(weights, degrees): 
    radians = np.deg2rad(degrees)
    x = np.cos(radians)
    y = np.sin(radians)
    mean_x = np.nansum(weights*x)
    mean_y = np.nansum(weights*y)
    mean_angle = np.arctan2(mean_y, mean_x)
    mean_angle = np.rad2deg(mean_angle)

    # 将坐标系从以正x为0°转换为以正y为0°
    mean_angle = (mean_angle + 90) % 360  # 将正y轴对齐到0°
    mean_angle = (360 - mean_angle) % 360  # 使顺时针方向为正
    mean_angle[mean_angle > 180] -= 360    # 转换到 [-180, 180) 范围

    adjusted_angle = np.where(mean_angle > 0, 180 - mean_angle, -180 - mean_angle)

    return  -adjusted_angle

In [178]:
work_dir = '/nfs/z1/userhome/zzl-xsk/Featurespace'
cifti_path = '/nfs/z1/userhome/zzl-xsk/NOD/derivatives/ciftify'
prf_dir = join(work_dir, 'build/retinoparams/primaryvis-in-MMP')
corrmap_dir = join(work_dir, 'build/corrmap')
guass_path = join(work_dir, 'build/gaussianparams/primaryvis-in-MMP')
voxel_mask_path = os.path.join(work_dir, 'prep/voxel_masks')
map_dir = join(work_dir, 'anal/brainmap/masked_retinotopy')
mask_name = 'primaryvis-in-MMP'
subs = [f'sub-0{isub}' for isub in range(1, 10)]

for sub in subs:
    #体素选择
    voxel_mask_nii = nib.load(os.path.join(voxel_mask_path, f'nod-voxmask_{mask_name}.dlabel.nii'))
    voxel_mask = voxel_mask_nii.get_fdata()
    prf_data = nib.load(os.path.join(cifti_path, f'{sub}/results/ses-prf_task-prf/ses-prf_task-prf_params.dscalar.nii')).get_fdata()
    #选取R2大于10的体素
    R2_values = prf_data[3, :]
    valid_R2_indices = np.where(R2_values >= 10)[0]
    mmp_voxel_mask = np.squeeze(np.array(voxel_mask))
    # 确定最终的mask indices
    mmp_voxel_indices = np.where(mmp_voxel_mask==1)[0]
    voxel_indices = np.intersect1d(mmp_voxel_indices, valid_R2_indices)
    #读取dnn prf 数值
    conv2_prf = np.load(join(prf_dir, f'{sub}_layer-googlenet-conv2_params.npy'),allow_pickle=True).item()
    conv2_prf_R2 = conv2_prf['R2']
    inception3a_prf = np.load(join(prf_dir, f'{sub}_layer-googlenet-inception3a_params.npy'),allow_pickle=True).item()
    inception3a_prf_R2 = inception3a_prf['R2']
    maxpool2_prf = np.load(join(prf_dir, f'{sub}_layer-googlenet-maxpool2_params.npy'),allow_pickle=True).item()
    maxpool2_prf_R2 = maxpool2_prf['R2']
    #
    new_prf = np.nan * np.zeros((4, 59412))
    new_guass = { }
    conv2_guass = np.load(join(guass_path, f'{sub}_layer-googlenet-conv2_Gauss.npy'),allow_pickle=True).item()
    inception3a_guass = np.load(join(guass_path, f'{sub}_layer-googlenet-inception3a_Gauss.npy'),allow_pickle=True).item()
    maxpool2_guass = np.load(join(guass_path, f'{sub}_layer-googlenet-maxpool2_Gauss.npy'),allow_pickle=True).item()

    for voxel_idx in voxel_indices:
        if voxel_idx not in conv2_guass.keys():
            conv2_guass[voxel_idx] = (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)
        if voxel_idx not in inception3a_guass.keys():
            inception3a_guass[voxel_idx] = (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)
        if voxel_idx not in maxpool2_guass.keys():
            maxpool2_guass[voxel_idx] = (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)
        
        r2_conv2 = conv2_prf_R2[voxel_idx]
        r2_inception3a = inception3a_prf_R2[voxel_idx]
        r2_maxpool2 = maxpool2_prf_R2[voxel_idx]

        r_weights = np.array([r2_conv2, r2_inception3a, r2_maxpool2])/np.nansum([r2_conv2, r2_inception3a, r2_maxpool2])
        
        eccs = np.array([conv2_prf['ecc'][voxel_idx], inception3a_prf['ecc'][voxel_idx], maxpool2_prf['ecc'][voxel_idx]])
        angs = np.array([conv2_prf['ang'][voxel_idx], inception3a_prf['ang'][voxel_idx], maxpool2_prf['ang'][voxel_idx]])
        rfsizes = np.array([conv2_prf['rfsize'][voxel_idx], inception3a_prf['rfsize'][voxel_idx], maxpool2_prf['rfsize'][voxel_idx]])
        r2s = np.array([conv2_prf['R2'][voxel_idx], inception3a_prf['R2'][voxel_idx], maxpool2_prf['R2'][voxel_idx]])


        As =  np.array([conv2_guass[voxel_idx][0],  inception3a_guass[voxel_idx][0], maxpool2_guass[voxel_idx][0]])
        xs =  np.array([conv2_guass[voxel_idx][1],  inception3a_guass[voxel_idx][1], maxpool2_guass[voxel_idx][1]])
        ys =  np.array([conv2_guass[voxel_idx][2],  inception3a_guass[voxel_idx][2], maxpool2_guass[voxel_idx][2]])
        xsigs =  np.array([conv2_guass[voxel_idx][3],  inception3a_guass[voxel_idx][3], maxpool2_guass[voxel_idx][3]])
        ysigs =  np.array([conv2_guass[voxel_idx][4],  inception3a_guass[voxel_idx][4], maxpool2_guass[voxel_idx][4]])
        Cs =  np.array([conv2_guass[voxel_idx][5],  inception3a_guass[voxel_idx][5], maxpool2_guass[voxel_idx][5]])

        A, x, y, C = np.nansum(r_weights * As), np.nansum(r_weights * xs), np.nansum(r_weights * ys), np.nansum(r_weights * Cs)
        xsig, ysig = np.sqrt(np.nansum(r_weights * xsigs ** 2)), np.sqrt(np.nansum(r_weights * ysigs ** 2)), 
        new_guass[voxel_idx] = (A, x , y, xsig, ysig, C) 


        new_prf[0, voxel_idx] = np.nansum(r_weights * eccs)
        new_prf[1, voxel_idx] = average_angle(r_weights, angs)
        new_prf[2, voxel_idx] = np.nansum(r_weights * rfsizes)
        new_prf[3, voxel_idx] = np.nanmean(r2s)

    named_maps = ['ecc', 'ang', 'rfsize', 'R2']
    bfiletemplate = nib.load('/nfs/z1/userhome/zzl-xsk/workingdir/template.dtseries.nii')
    brain_models = bfiletemplate.header.get_index_map(1).brain_models
    surface_brain_models = [bm for bm in brain_models if bm.model_type == 'CIFTI_MODEL_TYPE_SURFACE']
    print(pjoin(map_dir, sub, f'{sub}_masked-prior-prf.dscalar.nii'))
    save2cifti(join(map_dir, sub, f'{sub}_weighted-masked-dnn-prf.dscalar.nii'),  
                new_prf, surface_brain_models, map_names=named_maps)
    np.save(join(guass_path, f'{sub}_weighted_Gauss.npy'), np.array([new_guass]), allow_pickle=True)

<ipython-input-176-a2cf9cebb070>:3: RuntimeWarning: invalid value encountered in remainder
  degrees = (degrees - 90) % 360
